In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv(r'C:\Loan_Default_Prediction_MLOps\data\accepted_2007_to_2018Q4.csv')
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.isnull().sum()

id                             0
member_id                2260701
loan_amnt                     33
funded_amnt                   33
funded_amnt_inv               33
                          ...   
settlement_status        2226455
settlement_date          2226455
settlement_amount        2226455
settlement_percentage    2226455
settlement_term          2226455
Length: 151, dtype: int64

In [4]:
df.drop('member_id', axis=1)

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,truck driver,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,66310712,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,Information Systems Officer,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,68476807,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,Contract Specialist,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260696,88985880,40000.0,40000.0,40000.0,60 months,10.49,859.56,B,B3,Vice President,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2260697,88224441,24000.0,24000.0,24000.0,60 months,14.49,564.56,C,C4,Program Manager,...,NaN,NaN,Cash,Y,Mar-2019,ACTIVE,Mar-2019,10000.0,44.82,1.0
2260698,88215728,14000.0,14000.0,14000.0,60 months,14.49,329.33,C,C4,Customer Service Technician,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2260699,Total amount funded in policy code 1: 1465324575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       ...
       'hardship_payoff_balance_amount', 'hardship_last_payment_amount',
       'disbursement_method', 'debt_settlement_flag',
       'debt_settlement_flag_date', 'settlement_status', 'settlement_date',
       'settlement_amount', 'settlement_percentage', 'settlement_term'],
      dtype='object', length=151)

### 데이터 누수 방지를 위해서 
###### 'loan_amnt', 'term', 'int_rate', 'grade', 'emp_length', 'home_ownership', 'annual_inc', 'dti', 'loan_status' 컬럼들만 선택
###### 대출을 신청하는 시점(At the point of application)에 알 수 있는 정보
### train.py에서 선택한 컬럼들은 이러한 '미래 정보'를 원천적으로 차단하여, 오직 신청 시점의 정보만으로 공정하게 예측하도록 만들기 위함

```
상환 능력:

- annual_inc (연소득): 돈을 얼마나 버는가?

- dti (소득 대비 부채 비율): 버는 돈에 비해 빚이 얼마나 많은가?

신용 및 재정 상태:

- grade (대출 등급): Lending Club이 이미 평가한 리스크 등급은 어떠한가?

- home_ownership (주거 형태): 자가 소유 여부로 재정 안정성을 가늠할 수 있는가?

- emp_length (고용 기간): 직업적으로 얼마나 안정적인가?

대출의 특성:

- loan_amnt (대출 금액): 얼마나 큰돈을 빌리는가?

- term (기간), int_rate (이자율): 어떤 조건의 대출인가?
```

In [6]:
selected_columns = [
    'loan_amnt', 'term', 'int_rate', 'grade', 'emp_length',
    'home_ownership', 'annual_inc', 'dti', 'loan_status', 
    'verification_status', 'purpose', 'issue_d'
]

features_df = df[selected_columns].copy().dropna() # 선택된 컬럼들 중 null(NaN)인 행들 삭제

In [7]:
features_df.head()

,loan_amnt,term,int_rate,grade,emp_length,home_ownership,annual_inc,dti,loan_status,verification_status,purpose,issue_d
0,3600.0,36 months,13.99,C,10+ years,MORTGAGE,55000.0,5.91,Fully Paid,Not Verified,debt_consolidation,Dec-2015
1,24700.0,36 months,11.99,C,10+ years,MORTGAGE,65000.0,16.06,Fully Paid,Not Verified,small_business,Dec-2015
2,20000.0,60 months,10.78,B,10+ years,MORTGAGE,63000.0,10.78,Fully Paid,Not Verified,home_improvement,Dec-2015
3,35000.0,60 months,14.85,C,10+ years,MORTGAGE,110000.0,17.06,Current,Source Verified,debt_consolidation,Dec-2015
4,10400.0,60 months,22.45,F,3 years,MORTGAGE,104433.0,25.37,Fully Paid,Source Verified,major_purchase,Dec-2015


In [8]:
features_df.shape

(2113644, 12)

In [9]:
features_df.term.value_counts() # 36개월: 1493315, 60개월: 620329

term
36 months    1493315
60 months     620329
Name: count, dtype: int64

In [10]:
features_df['term'] = features_df['term'].str.replace(' months', '').astype(int) # months 버리고 36, 60을 int형으로 변환


In [11]:
features_df.head()

,loan_amnt,term,int_rate,grade,emp_length,home_ownership,annual_inc,dti,loan_status,verification_status,purpose,issue_d
0,3600.0,36,13.99,C,10+ years,MORTGAGE,55000.0,5.91,Fully Paid,Not Verified,debt_consolidation,Dec-2015
1,24700.0,36,11.99,C,10+ years,MORTGAGE,65000.0,16.06,Fully Paid,Not Verified,small_business,Dec-2015
2,20000.0,60,10.78,B,10+ years,MORTGAGE,63000.0,10.78,Fully Paid,Not Verified,home_improvement,Dec-2015
3,35000.0,60,14.85,C,10+ years,MORTGAGE,110000.0,17.06,Current,Source Verified,debt_consolidation,Dec-2015
4,10400.0,60,22.45,F,3 years,MORTGAGE,104433.0,25.37,Fully Paid,Source Verified,major_purchase,Dec-2015


In [12]:
features_df.grade.value_counts()

grade
B    620392
C    607554
A    405905
D    301765
E    127082
F     39436
G     11510
Name: count, dtype: int64

In [13]:
# grade 컬럼 label Encoding 실행

from sklearn.preprocessing import LabelEncoder

# label encoding
le = LabelEncoder()

# 인코딩 적용
features_df['grade'] = le.fit_transform(features_df['grade'])

features_df.head()

,loan_amnt,term,int_rate,grade,emp_length,home_ownership,annual_inc,dti,loan_status,verification_status,purpose,issue_d
0,3600.0,36,13.99,2,10+ years,MORTGAGE,55000.0,5.91,Fully Paid,Not Verified,debt_consolidation,Dec-2015
1,24700.0,36,11.99,2,10+ years,MORTGAGE,65000.0,16.06,Fully Paid,Not Verified,small_business,Dec-2015
2,20000.0,60,10.78,1,10+ years,MORTGAGE,63000.0,10.78,Fully Paid,Not Verified,home_improvement,Dec-2015
3,35000.0,60,14.85,2,10+ years,MORTGAGE,110000.0,17.06,Current,Source Verified,debt_consolidation,Dec-2015
4,10400.0,60,22.45,5,3 years,MORTGAGE,104433.0,25.37,Fully Paid,Source Verified,major_purchase,Dec-2015


In [14]:
features_df.home_ownership.value_counts()

home_ownership
MORTGAGE    1045383
RENT         845200
OWN          221991
ANY             844
OTHER           179
NONE             47
Name: count, dtype: int64

In [15]:
# 'home_ownership' One-Hot Encoding 실행
from sklearn.preprocessing import OneHotEncoder

features_df = pd.get_dummies(features_df, columns=['home_ownership'], prefix='home_ownership', dtype=int)
features_df.head()

,loan_amnt,term,int_rate,grade,emp_length,annual_inc,dti,loan_status,verification_status,purpose,issue_d,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT
0,3600.0,36,13.99,2,10+ years,55000.0,5.91,Fully Paid,Not Verified,debt_consolidation,Dec-2015,0,1,0,0,0,0
1,24700.0,36,11.99,2,10+ years,65000.0,16.06,Fully Paid,Not Verified,small_business,Dec-2015,0,1,0,0,0,0
2,20000.0,60,10.78,1,10+ years,63000.0,10.78,Fully Paid,Not Verified,home_improvement,Dec-2015,0,1,0,0,0,0
3,35000.0,60,14.85,2,10+ years,110000.0,17.06,Current,Source Verified,debt_consolidation,Dec-2015,0,1,0,0,0,0
4,10400.0,60,22.45,5,3 years,104433.0,25.37,Fully Paid,Source Verified,major_purchase,Dec-2015,0,1,0,0,0,0


In [16]:
features_df.home_ownership_ANY.value_counts()

home_ownership_ANY
0    2112800
1        844
Name: count, dtype: int64

In [17]:
features_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2113644 entries, 0 to 2260698
Data columns (total 17 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   loan_amnt                float64
 1   term                     int64  
 2   int_rate                 float64
 3   grade                    int64  
 4   emp_length               object 
 5   annual_inc               float64
 6   dti                      float64
 7   loan_status              object 
 8   verification_status      object 
 9   purpose                  object 
 10  issue_d                  object 
 11  home_ownership_ANY       int64  
 12  home_ownership_MORTGAGE  int64  
 13  home_ownership_NONE      int64  
 14  home_ownership_OTHER     int64  
 15  home_ownership_OWN       int64  
 16  home_ownership_RENT      int64  
dtypes: float64(4), int64(8), object(5)
memory usage: 290.3+ MB


In [18]:
# 'emp_length' -> 수동 매핑
# 명확한 순서가 있는 데이터이므로, 이 순서 정보를 최대한 살려서 숫자로 바꿔주는 것

emp_length_map = {
    '< 1 year': 0,
    '1 year': 1,
    '2 years': 2,
    '3 years': 3,
    '4 years': 4,
    '5 years': 5,
    '6 years': 6,
    '7 years': 7,
    '8 years': 8,
    '9 years': 9,
    '10+ years': 10
}

features_df['emp_length'] = df['emp_length'].map(emp_length_map)

In [19]:
features_df.head()

,loan_amnt,term,int_rate,grade,emp_length,annual_inc,dti,loan_status,verification_status,purpose,issue_d,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT
0,3600.0,36,13.99,2,10.0,55000.0,5.91,Fully Paid,Not Verified,debt_consolidation,Dec-2015,0,1,0,0,0,0
1,24700.0,36,11.99,2,10.0,65000.0,16.06,Fully Paid,Not Verified,small_business,Dec-2015,0,1,0,0,0,0
2,20000.0,60,10.78,1,10.0,63000.0,10.78,Fully Paid,Not Verified,home_improvement,Dec-2015,0,1,0,0,0,0
3,35000.0,60,14.85,2,10.0,110000.0,17.06,Current,Source Verified,debt_consolidation,Dec-2015,0,1,0,0,0,0
4,10400.0,60,22.45,5,3.0,104433.0,25.37,Fully Paid,Source Verified,major_purchase,Dec-2015,0,1,0,0,0,0


In [20]:
# loan_status가 완납이면 1, 미납이면 0

features_df['loan_status'] = (features_df['loan_status'] == 'Fully Paid').astype(int)
features_df.head()

,loan_amnt,term,int_rate,grade,emp_length,annual_inc,dti,loan_status,verification_status,purpose,issue_d,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT
0,3600.0,36,13.99,2,10.0,55000.0,5.91,1,Not Verified,debt_consolidation,Dec-2015,0,1,0,0,0,0
1,24700.0,36,11.99,2,10.0,65000.0,16.06,1,Not Verified,small_business,Dec-2015,0,1,0,0,0,0
2,20000.0,60,10.78,1,10.0,63000.0,10.78,1,Not Verified,home_improvement,Dec-2015,0,1,0,0,0,0
3,35000.0,60,14.85,2,10.0,110000.0,17.06,0,Source Verified,debt_consolidation,Dec-2015,0,1,0,0,0,0
4,10400.0,60,22.45,5,3.0,104433.0,25.37,1,Source Verified,major_purchase,Dec-2015,0,1,0,0,0,0


In [21]:
features_df.loan_status.value_counts()

loan_status
0    1094288
1    1019356
Name: count, dtype: int64

In [22]:
purpose1 = features_df.purpose.value_counts()

In [23]:
# 신뢰도를 위해 전체의 10%보다 크기가 작은 값들은 others에 묶음
threshold = len(df) * 0.1
rare_purposes = purpose1[purpose1 < threshold].index

features_df['purpose_cleaned'] = features_df['purpose'].replace(rare_purposes, 'others')

In [24]:
features_df.head()

,loan_amnt,term,int_rate,grade,emp_length,annual_inc,dti,loan_status,verification_status,purpose,issue_d,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,purpose_cleaned
0,3600.0,36,13.99,2,10.0,55000.0,5.91,1,Not Verified,debt_consolidation,Dec-2015,0,1,0,0,0,0,debt_consolidation
1,24700.0,36,11.99,2,10.0,65000.0,16.06,1,Not Verified,small_business,Dec-2015,0,1,0,0,0,0,others
2,20000.0,60,10.78,1,10.0,63000.0,10.78,1,Not Verified,home_improvement,Dec-2015,0,1,0,0,0,0,others
3,35000.0,60,14.85,2,10.0,110000.0,17.06,0,Source Verified,debt_consolidation,Dec-2015,0,1,0,0,0,0,debt_consolidation
4,10400.0,60,22.45,5,3.0,104433.0,25.37,1,Source Verified,major_purchase,Dec-2015,0,1,0,0,0,0,others


In [25]:
# verification_status -> one-hot encoding
# 적용 이유: 모델이 스스로 각 상태의 중요도를 학습하게 두는 것이 가장 강력하고 일반적인 접근 방식이기 때문

features_df = pd.get_dummies(features_df, columns=['verification_status'], prefix='verification_status', dtype=int)

In [26]:
# 카테고리는 많지만 분포가 불균등하고 순서가 없는 경우
# 먼저 데이터의 분포를 보고 희귀 값을 정리한 뒤
# 원-핫 인코딩을 적용하는 것이 모델의 성능과 안정성을 모두 잡는 가장 정석적인 전처리 방법

features_df = pd.get_dummies(features_df, columns=['purpose_cleaned'], prefix='purpose', dtype=int)

In [27]:
features_df = features_df.drop('purpose', axis=1)

In [28]:
features_df.head()

,loan_amnt,term,int_rate,grade,emp_length,annual_inc,dti,loan_status,issue_d,home_ownership_ANY,...,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,purpose_credit_card,purpose_debt_consolidation,purpose_others
0,3600.0,36,13.99,2,10.0,55000.0,5.91,1,Dec-2015,0,...,0,0,0,0,1,0,0,0,1,0
1,24700.0,36,11.99,2,10.0,65000.0,16.06,1,Dec-2015,0,...,0,0,0,0,1,0,0,0,0,1
2,20000.0,60,10.78,1,10.0,63000.0,10.78,1,Dec-2015,0,...,0,0,0,0,1,0,0,0,0,1
3,35000.0,60,14.85,2,10.0,110000.0,17.06,0,Dec-2015,0,...,0,0,0,0,0,1,0,0,1,0
4,10400.0,60,22.45,5,3.0,104433.0,25.37,1,Dec-2015,0,...,0,0,0,0,0,1,0,0,0,1


### 그룹 1: 수치형 정보 (Numeric Features)
대출의 조건과 신청자의 재정 상태를 직접적으로 나타내는 숫자 값들

- loan_amnt (대출 신청 금액): 신청자가 빌리고자 하는 원금. 금액이 클수록 리스크가 커질 수 있음

- term (대출 기간): 대출 상환 기간을 '개월' 단위의 숫자로 변환한 값(e.g., 36, 60). 기간이 길수록 불확실성이 커져 리스크가 증가할 수 있음

- int_rate (이자율): 대출에 적용되는 이자율. 일반적으로 신용도가 낮을수록 이자율이 높게 책정됨

- annual_inc (연간 소득): 신청자의 연간 소득. 상환 능력을 나타내는 가장 직접적인 지표

- dti (소득 대비 부채 비율): 신청자의 월 소득에서 월별 부채 상환금이 차지하는 비율. 이 비율이 높을수록 가처분 소득이 적어 상환 부담이 큼

### 그룹 2: 순서형 정보 (Ordinal Features)
카테고리 간에 명확한 순서나 등급이 있어, 이를 숫자로 변환한 정보

- grade (대출 등급): Lending Club이 평가한 신용 등급을 숫자로 변환한 값(e.g., A=0, B=1, ..., G=6). 숫자가 클수록 리스크가 높다는 순서 정보를 모델에게 전달

- emp_length (고용 기간): 신청자의 고용 기간을 숫자로 변환한 값(e.g., < 1 year=0, ..., 10+ years=10). 숫자가 클수록 직업적 안정성이 높다고 판단

### 그룹 3: 타겟 변수 (Target Variable)
우리가 최종적으로 예측하려는 목표 값

- loan_status (대출 상태): 대출의 최종 결과를 나타냄. 1은 'Charged Off'(부도), **0은 'Fully Paid'(완납)**를 의미. 모델은 다른 모든 피처들을 보고 이 값을 맞추는 것을 학습

### 그룹 4: 명목형 정보 (One-Hot Encoded Features)
순서가 없는 카테고리들을 0 또는 1의 값을 갖는 여러 개의 컬럼으로 변환한 정보. 특정 카테고리에 해당하면 1, 아니면 0

- home_ownership_* (주거 소유 형태):

    - home_ownership_MORTGAGE: 주택 담보대출을 통해 집을 소유하고 있는지 여부

    - home_ownership_RENT: 집을 임차하고 있는지 여부

    - home_ownership_OWN: 대출 없이 집을 완전히 소유하고 있는지 여부

(그 외 ANY, NONE, OTHER 등)

- verification_status_* (소득 증명 상태):

    - verification_status_Not Verified: 소득 증빙 서류를 확인하지 않았는지 여부

    - verification_status_Source Verified: 소득의 출처까지 확인했는지 여부

    - verification_status_Verified: 소득 증빙 서류를 확인했는지 여부

- purpose_* (대출 목적):

    - purpose_credit_card: 신용카드 대금 상환이 목적인지 여부

    - purpose_debt_consolidation: 다른 부채를 통합하는 것이 목적인지 여부

    - (그 외 home_improvement, car 등 다양한 목적들)

In [29]:
features_df.columns

Index(['loan_amnt', 'term', 'int_rate', 'grade', 'emp_length', 'annual_inc',
       'dti', 'loan_status', 'issue_d', 'home_ownership_ANY',
       'home_ownership_MORTGAGE', 'home_ownership_NONE',
       'home_ownership_OTHER', 'home_ownership_OWN', 'home_ownership_RENT',
       'verification_status_Not Verified',
       'verification_status_Source Verified', 'verification_status_Verified',
       'purpose_credit_card', 'purpose_debt_consolidation', 'purpose_others'],
      dtype='object')

In [30]:
features_df.issue_d.value_counts()

issue_d
Mar-2016    57586
Oct-2015    45712
Jul-2015    43496
May-2018    42608
Aug-2018    42240
            ...  
Aug-2007       72
Jul-2007       62
Sep-2008       57
Sep-2007       53
Jun-2007       23
Name: count, Length: 139, dtype: int64

In [ ]:
# issue_d를 datetime으로 변환
features_df["issue_d"] = pd.to_datetime(features_df["issue_d"], format="%b-%Y")

# 연도만 추출
features_df["issue_year"] = features_df["issue_d"].dt.year

# 원하는 연도만 남기기
valid_years = [2013, 2014, 2015, 2016, 2017, 2018]
features_df = features_df[features_df["issue_year"].isin(valid_years)]

# 필요 없다면 issue_year 제거
features_df = features_df.drop(columns=["issue_year"])

In [32]:
features_df.head()

,loan_amnt,term,int_rate,grade,emp_length,annual_inc,dti,loan_status,issue_d,home_ownership_ANY,...,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,purpose_credit_card,purpose_debt_consolidation,purpose_others
0,3600.0,36,13.99,2,10.0,55000.0,5.91,1,2015-12-01,0,...,0,0,0,0,1,0,0,0,1,0
1,24700.0,36,11.99,2,10.0,65000.0,16.06,1,2015-12-01,0,...,0,0,0,0,1,0,0,0,0,1
2,20000.0,60,10.78,1,10.0,63000.0,10.78,1,2015-12-01,0,...,0,0,0,0,1,0,0,0,0,1
3,35000.0,60,14.85,2,10.0,110000.0,17.06,0,2015-12-01,0,...,0,0,0,0,0,1,0,0,1,0
4,10400.0,60,22.45,5,3.0,104433.0,25.37,1,2015-12-01,0,...,0,0,0,0,0,1,0,0,0,1


In [33]:
# 학습데이터(2013,2014,2015년도 데이터), 검증데이터(2016, 2017년도), 모니터링 데이터(2018년도)로 train_df, valid_df, test_df로 나눔


# 연도 컬럼 생성
features_df["issue_year"] = features_df["issue_d"].dt.year

# Split
train_df = features_df[features_df["issue_year"].isin([2013, 2014, 2015])]
valid_df = features_df[features_df["issue_year"].isin([2016, 2017])]
test_df  = features_df[features_df["issue_year"] == 2018]

In [52]:
train_df.issue_d.value_counts()

issue_d
2015-10-01    45712
2015-07-01    43496
2015-12-01    41550
2014-10-01    36722
2015-11-01    35210
2015-04-01    33664
2015-08-01    33553
2015-01-01    33286
2015-05-01    30310
2014-07-01    28103
2015-06-01    27073
2015-09-01    27065
2015-03-01    24051
2014-11-01    23655
2015-02-01    22308
2014-05-01    18206
2014-04-01    18061
2014-08-01    17888
2014-06-01    16405
2014-03-01    15644
2014-01-01    14775
2014-02-01    14347
2013-12-01    14315
2013-11-01    14066
2013-10-01    13407
2013-09-01    12446
2013-08-01    12063
2013-07-01    11344
2013-06-01    10407
2014-09-01    10075
2013-05-01     9853
2014-12-01     9729
2013-04-01     9039
2013-03-01     7976
2013-02-01     7288
2013-01-01     6648
Name: count, dtype: int64

In [47]:
valid_df.issue_d.value_counts()

issue_d
2016-03-01    57586
2017-08-01    40558
2017-11-01    38802
2017-07-01    36930
2016-02-01    36930
2017-09-01    36546
2017-06-01    35845
2017-05-01    35261
2017-10-01    35170
2017-12-01    34935
2017-03-01    34533
2016-08-01    34112
2016-04-01    33736
2016-12-01    33490
2016-07-01    32646
2016-11-01    32369
2016-06-01    31239
2016-10-01    30807
2016-01-01    30427
2017-01-01    29722
2017-04-01    27645
2016-09-01    26440
2016-05-01    26408
2017-02-01    25730
Name: count, dtype: int64

In [49]:
test_df.issue_d.value_counts()

issue_d
2018-05-01    42608
2018-08-01    42240
2018-10-01    42177
2018-07-01    39910
2018-04-01    39484
2018-06-01    38185
2018-11-01    38071
2018-12-01    36438
2018-09-01    35639
2018-03-01    35290
2018-01-01    33413
2018-02-01    29721
Name: count, dtype: int64

In [35]:
X_train = train_df.drop(["issue_d", "loan_status"], axis=1)
y_train = train_df["loan_status"]

X_valid = valid_df.drop(["issue_d", "loan_status"], axis=1)
y_valid = valid_df["loan_status"]

X_test  = test_df.drop(["issue_d", "loan_status"], axis=1)
y_test  = test_df["loan_status"]

In [ ]:
# LightGBM

import lightgbm as lgb
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score, f1_score
import warnings
import mlflow

# 경고 메시지 무시
warnings.filterwarnings('ignore')

print("\n. LightGBM 모델 학습 및 평가 시작")

# 모델 정의 및 학습
lgbm_model = lgb.LGBMClassifier(random_state=42)
lgbm_model.fit(X_train, y_train)

# 검증 데이터 예측
val_proba = lgbm_model.predict_proba(X_valid)[:, 1]   # 확률 값
val_pred = lgbm_model.predict(X_valid)                # 클래스 값

# 성능 평가 지표 계산
auc_score = roc_auc_score(y_valid, val_proba)
precision = precision_score(y_valid, val_pred)
recall = recall_score(y_valid, val_pred)
accuracy = accuracy_score(y_valid, val_pred)
f1 = f1_score(y_valid, val_pred)

# 결과 출력
print(f"학습 및 평가 완료")
print(f"AUC       = {auc_score:.4f}")
print(f"Precision = {precision:.4f}")
print(f"Recall    = {recall:.4f}")
print(f"Accuracy  = {accuracy:.4f}")
print(f"F1-score  = {f1:.4f}")

2025/09/16 19:14:03 INFO mlflow.tracking.fluent: Experiment with name 'Loan_Deafault_Prediction' does not exist. Creating a new experiment.
2025/09/16 19:14:03 INFO mlflow.tracking.fluent: Autologging successfully enabled for lightgbm.
2025/09/16 19:14:04 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/09/16 19:14:05 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '0b5fff4d555b4ac8a2922391a8eb0c7c', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current lightgbm workflow



. LightGBM 모델 학습 및 평가 시작


2025/09/16 19:14:12 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run efficient-pig-218 at: http://127.0.0.1:5000/#/experiments/18/runs/0b5fff4d555b4ac8a2922391a8eb0c7c
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/18
학습 및 평가 완료
AUC       = 0.5944
Precision = 0.4461
Recall    = 0.8563
Accuracy  = 0.5074
F1-score  = 0.5866


In [ ]:
# LogisticRegression + StandardScaler

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, precision_score, recall_score

numeric_cols = ["loan_amnt", "term", "int_rate", "grade", 
                "emp_length", "annual_inc", "dti"]

sc = StandardScaler()
lr = LogisticRegression(class_weight="balanced", random_state=42)

# 스케일링 적용
X_train_scaled = X_train.copy()
X_valid_scaled = X_valid.copy()
X_test_scaled  = X_test.copy()

X_train_scaled[numeric_cols] = sc.fit_transform(X_train[numeric_cols])
X_valid_scaled[numeric_cols] = sc.transform(X_valid[numeric_cols])
X_test_scaled[numeric_cols]  = sc.transform(X_test[numeric_cols])

# 모델 학습
lr.fit(X_train_scaled, y_train)

# 예측
y_pred_valid = lr.predict(X_valid_scaled)                # 클래스 (0/1)
y_proba_valid = lr.predict_proba(X_valid_scaled)[:, 1]   # 확률 (0~1)

# 성능 평가 지표
accuracy  = accuracy_score(y_valid, y_pred_valid)
precision = precision_score(y_valid, y_pred_valid)
recall    = recall_score(y_valid, y_pred_valid)
f1        = f1_score(y_valid, y_pred_valid)
auc_score       = roc_auc_score(y_valid, y_proba_valid)

# 결과 출력
print(f"학습 및 평가 완료")
print(f"Accuracy  : {accuracy:.4f}")
print(f"Precision : {precision:.4f}")
print(f"Recall    : {recall:.4f}")
print(f"F1-score  : {f1:.4f}")
print(f"AUC       : {auc_score:.4f}")


학습 및 평가 완료
Accuracy  : 0.5315
Precision : 0.4570
Recall    : 0.7858
F1-score  : 0.5779
AUC       : 0.5933


In [ ]:
# LogisticRegression
# 부실 대출을 막기 위해선 스케일링 안한 LogisticRegression 모델 적용 -> recall이 제일 높음

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, accuracy_score

lr = LogisticRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_valid)
precision = precision_score(y_valid, y_pred)
recall = recall_score(y_valid, y_pred)
f1 = f1_score(y_valid, y_pred)
print(f"Precision: {precision}")
print(f"Recall Score: {recall}")
print(f"F1-Score: {f1}")

accuracy = accuracy_score(y_valid, y_pred)

print(f"accuracy: {accuracy}")

val_proba = lr.predict_proba(X_valid)[:, 1]
auc_score = roc_auc_score(y_valid, val_proba)
print(f"AUC: {auc_score}")

Precision: 0.42252079334612924
Recall Score: 0.9296672357504417
F1-Score: 0.580989817310572
accuracy: 0.45259559317101683
AUC: 0.592585361448794


In [44]:
## LightGBM + Optuna 

import optuna
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score

def objective(trial):
    # LightGBM 하이퍼파라미터 샘플링
    param = {
        "objective": "binary",
        "metric": "auc",
        "boosting_type": "gbdt",
        "verbosity": -1,
        "random_state": 42,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 16, 256),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_child_samples": trial.suggest_int("min_child_samples", 10, 200),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
    }
    
    # 모델 정의 및 학습
    model = lgb.LGBMClassifier(**param, n_estimators=1000)
    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        eval_metric="auc"
    )
    
    # 검증 AUC 반환
    val_pred = model.predict_proba(X_valid)[:, 1]
    auc_score = roc_auc_score(y_valid, val_pred)
    return auc_score

# Optuna 실행
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

# 최적 파라미터 확인
print("Best trial:")
print(study.best_trial.params)

# 최적 파라미터로 최종 모델 학습
best_params = study.best_trial.params
best_model = lgb.LGBMClassifier(**best_params, n_estimators=1000, random_state=42)
best_model.fit(
    X_train, y_train,
    eval_set=[(X_valid, y_valid)],
    eval_metric="auc",
    early_stopping_rounds=3,
    verbose=100
)

# 최종 성능 확인
y_pred_valid = best_model.predict(X_valid)                # 클래스
y_proba_valid = best_model.predict_proba(X_valid)[:, 1]   # 확률

accuracy  = accuracy_score(y_valid, y_pred_valid)
precision = precision_score(y_valid, y_pred_valid)
recall    = recall_score(y_valid, y_pred_valid)
f1        = f1_score(y_valid, y_pred_valid)
auc       = roc_auc_score(y_valid, y_proba_valid)

print(f"학습 및 평가 완료")
print(f"Accuracy  : {accuracy:.4f}")
print(f"Precision : {precision:.4f}")
print(f"Recall    : {recall:.4f}")
print(f"F1-score  : {f1:.4f}")
print(f"AUC       : {auc:.4f}")


[I 2025-09-16 17:24:10,770] A new study created in memory with name: no-name-f5824c76-4be7-4460-b107-b255cfdd2e84
[I 2025-09-16 17:25:28,184] Trial 0 finished with value: 0.5922513734368956 and parameters: {'learning_rate': 0.052119690965513756, 'num_leaves': 31, 'max_depth': 10, 'min_child_samples': 189, 'subsample': 0.9651861785820464, 'colsample_bytree': 0.6299967940869987, 'reg_alpha': 4.3022608940688256e-07, 'reg_lambda': 5.102390591971742e-07}. Best is trial 0 with value: 0.5922513734368956.
[I 2025-09-16 17:27:13,772] Trial 1 finished with value: 0.5911267762764988 and parameters: {'learning_rate': 0.0487405666012923, 'num_leaves': 103, 'max_depth': 9, 'min_child_samples': 136, 'subsample': 0.7523913794463962, 'colsample_bytree': 0.6333491539736741, 'reg_alpha': 3.790437002118492e-07, 'reg_lambda': 0.23613128143715773}. Best is trial 0 with value: 0.5922513734368956.
[I 2025-09-16 17:28:58,247] Trial 2 finished with value: 0.5929018789848612 and parameters: {'learning_rate': 0.0

KeyboardInterrupt: 

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 원본 CSV 로드 (필요한 컬럼만)
df = pd.read_csv("C:\Loan_Default_Prediction_MLOps\data\loan_approval_dataset_2013plus.csv", low_memory=True)

# 총 200만 개 목표
target_size = 2_000_000

# 연도별 데이터셋 크기 확인
year_counts = df["issue_year"].value_counts().sort_index()
print("연도별 데이터 크기:\n", year_counts)

# 연도별 샘플링 비율 계산
year_ratios = year_counts / year_counts.sum()
year_samples = (year_ratios * target_size).astype(int)
print("연도별 샘플링 목표 개수:\n", year_samples)

# Stratified Sampling 적용
sampled_list = []
for year, n_samples in year_samples.items():
    df_year = df[df["issue_year"] == year]
    if n_samples > len(df_year):
        n_samples = len(df_year)  # 안전 장치
    
    # stratify=target → 승인/거절 비율 유지
    sampled, _ = train_test_split(
        df_year,
        train_size=n_samples,
        stratify=df_year["target"],
        random_state=42
    )
    sampled_list.append(sampled)

# 최종 병합
df_sampled = pd.concat(sampled_list, ignore_index=True)
print("최종 샘플 데이터 크기:", df_sampled.shape)

# CSV 저장
df_sampled.to_csv("loan_approval_2M_stratified.csv", index=False)


<>:5: SyntaxWarning: invalid escape sequence '\L'
<>:5: SyntaxWarning: invalid escape sequence '\L'
C:\Users\yoodongseok\AppData\Local\Temp\ipykernel_15924\3590538138.py:5: SyntaxWarning: invalid escape sequence '\L'
  df = pd.read_csv("C:\Loan_Default_Prediction_MLOps\data\loan_approval_dataset_2013plus.csv", low_memory=True)


연도별 데이터 크기:
 issue_year
2013     895756
2014    2169329
2015    3280474
2016    5204281
2017    7516152
2018    9992024
Name: count, dtype: int64
연도별 샘플링 목표 개수:
 issue_year
2013     61652
2014    149310
2015    225787
2016    358199
2017    517320
2018    687729
Name: count, dtype: int64
최종 샘플 데이터 크기: (1999997, 13)


In [2]:
df1 = pd.read_csv("C:\Loan_Default_Prediction_MLOps\data\loan_approval_2M_stratified.csv")
df1.head()

<>:1: SyntaxWarning: invalid escape sequence '\L'
<>:1: SyntaxWarning: invalid escape sequence '\L'
C:\Users\yoodongseok\AppData\Local\Temp\ipykernel_15924\2725720369.py:1: SyntaxWarning: invalid escape sequence '\L'
  df1 = pd.read_csv("C:\Loan_Default_Prediction_MLOps\data\loan_approval_2M_stratified.csv")


,amount_requested,employment_length,application_date,loan_title,zip_code,state,dti,policy_code,risk_score,target,issue_year,issue_month,issue_quarter
0,20000.0,< 1 year,2013-06-24,debt_consolidation,926xx,CA,13.29,0.0,643.0,0,2013,6,2
1,10000.0,3 years,2013-09-01,home_improvement,548xx,WI,5.63,1.0,NaN,1,2013,9,3
2,16000.0,10+ years,2013-06-01,home_improvement,152xx,PA,16.74,1.0,NaN,1,2013,6,2
3,25000.0,< 1 year,2013-06-04,major_purchase,212xx,MD,0.00,0.0,0.0,0,2013,6,2
4,1500.0,3 years,2013-09-27,major_purchase,346xx,FL,23.78,0.0,599.0,0,2013,9,3
